In [1]:
import pandas as pd
frame1=pd.read_csv('train.tsv',sep='\t')

In [2]:
import nltk
import string
import os

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

path = '/opt/datacourse/data/parts'
token_dict = {}
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(wordnet_lemmatizer.lemmatize(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems
i=0
for line in frame1['Phrase']:
    
    lowers = line.lower()
    no_punctuation = lowers.translate(str.maketrans('','',string.punctuation))
    token_dict[i] = no_punctuation
    i=i+1    
#this can take some time


In [3]:
senti=list(frame1['Sentiment'])
values=list(token_dict.values())

In [4]:
frame2=pd.DataFrame({'Phrases':values,'Sentiment':senti})

In [5]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

In [6]:
feature_names = tfidf.get_feature_names()
px=[]
b=dict()
for x in tfs:
    a=[]
    for col in x.nonzero()[1]:
        if(x[0,col]>0.15):
            #print(col)
            #print (feature_names[col], ' - ', x[0, col])
            a.append(feature_names[col])
        else:
            if feature_names[col] not in b:
                b[feature_names[col]]=x[0,col]
            b[feature_names[col]]=min(x[0,col],b[feature_names[col]])
    px.append(a)
b

{'action': 0.1445007898801184,
 'best': 0.1479328102303952,
 'better': 0.1439743187885415,
 'character': 0.12361144847968523,
 'come': 0.13390255742941914,
 'comedy': 0.1361141087988782,
 'director': 0.14072808646427404,
 'doe': 0.136165459107376,
 'far': 0.1371269277109505,
 'feel': 0.1409638196904079,
 'film': 0.09607206743522378,
 'filmmaker': 0.13939071314895493,
 'funny': 0.14447537520480894,
 'good': 0.12993798097751988,
 'ha': 0.10377060764276107,
 'hollywood': 0.14874331752081388,
 'humor': 0.12985140924662866,
 'just': 0.1335540234181871,
 'life': 0.13018428612746183,
 'like': 0.11285269715565506,
 'little': 0.1364790740238406,
 'look': 0.13887246868038747,
 'love': 0.13951314725779473,
 'lrb': 0.12024900717467459,
 'make': 0.1197929035665218,
 'moment': 0.14981882686086284,
 'movie': 0.09136924140208673,
 'new': 0.144029548113106,
 'nt': 0.1103129720349559,
 'people': 0.14588091828565622,
 'performance': 0.12429261558172267,
 'real': 0.14633861892377573,
 'rrb': 0.11677977250

In [7]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

for i in range(len(px)):
    temp=px[i]
    px[i]=[]
    for w in temp:
        if w not in stop_words:
            px[i].append(w)

for i in range(len(px)):
    px[i]=' '.join(px[i])


In [8]:
frame3=pd.read_csv('frame3.csv')

In [21]:
max_words=96
px=[]
px=frame3['phrase']
f=open('phrase.csv')
px=f.readlines()
f.close()
'''    len1=len(px[i])
    if len1>max_words:
        max_words=len1
'''
vocab_size=len(feature_names)   

In [152]:
import tensorflow as tf
num_units=128
num_classes=5
embedding_size=128
with tf.variable_scope('sentiment',reuse=tf.AUTO_REUSE):
    lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(num_units)
    px_input=tf.placeholder(shape=(None,max_words),dtype=tf.int32)
    labels=tf.placeholder(shape=(None),dtype=tf.int32)
    embedding_encoder=tf.get_variable("embedding_encoder",shape=[vocab_size,embedding_size])
    encoder_inp = tf.nn.embedding_lookup(
        embedding_encoder, px_input)

    output,state=tf.nn.dynamic_rnn(lstm_cell,encoder_inp,time_major=False,dtype=tf.float32)
    output1=output[:,-1,:]
    weights=tf.Variable(tf.random_normal(shape=([num_units,num_classes])),dtype=tf.float32)
    bias=tf.Variable(tf.random_normal([num_classes]),dtype=tf.float32)
    output_final=output1 @ weights + bias
    logits=tf.nn.softmax(output_final)

In [160]:
batch_size=1000
learning_rate=0.1
cross_ent=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
loss=tf.reduce_sum(cross_ent)/batch_size
logits_arg=tf.argmax(logits,axis=1)
accuracy=tf.metrics.accuracy(labels,logits_arg)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [91]:
from keras.preprocessing.text import Tokenizer
t=Tokenizer()
t.fit_on_texts(px)
train_X=t.texts_to_sequences(px)

In [104]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np
frame3=pd.read_csv('frame3.csv')
train_X= np.array(pad_sequences(train_X,maxlen=max_words,padding='post'))
type(train_X)

from keras.utils import to_categorical
y_train=np.array(frame3['sentiment'])
#y_train=to_categorical(y_train)
length=y_train.shape
length=int(length[0])

In [143]:
from sklearn.utils import shuffle
  
ratio={0:12000,1:30000,2:79582,3:32927,4:15000}
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42,ratio=ratio)
train_X,y_train = sm.fit_sample(train_X,y_train)

train_X,y_train=shuffle(train_X,y_train)

In [164]:

init = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
px=np.array(px)
epochs=3
saver=tf.train.Saver()
save_path='E:\kaggle'
with tf.Session() as sess:
    sess.run(init)
    sess.run(init_l)
    #saver.restore(sess,save_path)
    for i in range(epochs):
        for j in range(int(5)):
            encoder_inp1=sess.run([encoder_inp],feed_dict={px_input:train_X[j*batch_size:(j+1)*batch_size],labels:y_train[j*batch_size:(j+1)*batch_size]})
            print("encoder:",encoder_inp1[0])
        #acc=sess.run(accuracy,feed_dict={px_input:train_X[j*batch_size:(j+1)*batch_size],labels:y_train[j*batch_size:(j+1)*batch_size]})
        #print(acc)
    saver.save(sess,save_path)

encoder: [[[ 2.4488606e-03 -1.1719112e-02  1.3949057e-02 ... -1.2023197e-02
   -9.4841467e-03 -1.8158911e-02]
  [ 6.3527562e-04 -1.3790283e-02  6.5411478e-03 ...  5.9429333e-03
   -1.1505699e-02 -1.1810152e-02]
  [ 6.3527562e-04 -1.3790283e-02  6.5411478e-03 ...  5.9429333e-03
   -1.1505699e-02 -1.1810152e-02]
  ...
  [ 6.3527562e-04 -1.3790283e-02  6.5411478e-03 ...  5.9429333e-03
   -1.1505699e-02 -1.1810152e-02]
  [ 6.3527562e-04 -1.3790283e-02  6.5411478e-03 ...  5.9429333e-03
   -1.1505699e-02 -1.1810152e-02]
  [ 6.3527562e-04 -1.3790283e-02  6.5411478e-03 ...  5.9429333e-03
   -1.1505699e-02 -1.1810152e-02]]

 [[ 6.4265821e-03  1.4378881e-02 -1.4843848e-03 ... -1.0606377e-02
    1.6011572e-02  1.4001580e-02]
  [ 1.6231308e-02 -1.9738190e-02 -8.8047655e-03 ...  1.4399288e-02
   -9.7757885e-03  7.6951664e-03]
  [ 1.9901605e-02 -3.2799561e-03 -3.3649765e-03 ...  1.5053751e-02
   -7.8533310e-03  1.3311388e-02]
  ...
  [ 6.3527562e-04 -1.3790283e-02  6.5411478e-03 ...  5.9429333e-03
 

encoder: [[[-0.01796528  0.01157017 -0.00583206 ... -0.01836419 -0.00843583
   -0.00910379]
  [-0.00975996 -0.00069585 -0.00276521 ...  0.01807731  0.0022929
   -0.01248616]
  [-0.00845558  0.00209542 -0.01569866 ...  0.0198625  -0.00053914
    0.00460164]
  ...
  [ 0.00063528 -0.01379028  0.00654115 ...  0.00594293 -0.0115057
   -0.01181015]
  [ 0.00063528 -0.01379028  0.00654115 ...  0.00594293 -0.0115057
   -0.01181015]
  [ 0.00063528 -0.01379028  0.00654115 ...  0.00594293 -0.0115057
   -0.01181015]]

 [[-0.01241433  0.01512991  0.01813888 ... -0.00447614 -0.00973139
    0.00381571]
  [ 0.00063528 -0.01379028  0.00654115 ...  0.00594293 -0.0115057
   -0.01181015]
  [ 0.00063528 -0.01379028  0.00654115 ...  0.00594293 -0.0115057
   -0.01181015]
  ...
  [ 0.00063528 -0.01379028  0.00654115 ...  0.00594293 -0.0115057
   -0.01181015]
  [ 0.00063528 -0.01379028  0.00654115 ...  0.00594293 -0.0115057
   -0.01181015]
  [ 0.00063528 -0.01379028  0.00654115 ...  0.00594293 -0.0115057
   -0.

In [159]:

#init = tf.global_variables_initializer()
#init_l = tf.local_variables_initializer()
px=np.array(px)
epochs=3
saver=tf.train.Saver()
save_path='E:\kaggle'
with tf.Session() as sess:
 #   sess.run(init)
  #  sess.run(init_l)
    saver.restore(sess,save_path)
    for i in range(1):
        for j in range(2):
            loss1,optimizer1=sess.run([loss,optimizer],feed_dict={px_input:train_X[j*batch_size:(j+1)*batch_size],labels:y_train[j*batch_size:(j+1)*batch_size]})
            print("epoch:",i,"batch:",j,"loss:",loss1)
        #acc=sess.run(accuracy,feed_dict={px_input:train_X[j*batch_size:(j+1)*batch_size],labels:y_train[j*batch_size:(j+1)*batch_size]})
        #print(acc)
    saver.save(sess,save_path)

INFO:tensorflow:Restoring parameters from E:\kaggle
epoch: 0 batch: 0 loss: 1.4538313
epoch: 0 batch: 1 loss: 1.4458314


In [97]:
logits1

array([[0.28545058, 0.11089214, 0.06655911, 0.29273295, 0.24436516],
       [0.28545055, 0.11089211, 0.06655909, 0.2927331 , 0.2443652 ]],
      dtype=float32)

In [99]:
loss1

0.0029917066

In [134]:
x='2 2'

In [135]:
len(x)

3